In [6]:
import canalystii
import struct
import json


In [7]:
dev = canalystii.CanalystDevice(bitrate=500000)

ValueError: No Canalyst-II USB device found

In [25]:
class Message:
    def __init__(self, chuck, checksum : int, counter : int):
        self.chuck = chuck
        self.checksum = checksum
        self.counter = counter
    def get_struct(self):
        #covert chuck to bytes
        if type(self.chuck) == str:
            #convert to ascii
            self.chuck = self.chuck.encode('ascii')
        return struct.pack('4sHH', self.chuck, self.checksum, self.counter)
    
    @staticmethod
    def fromBytesToMessage(raw_message):
        return Message(*struct.unpack('4sHH', bytes(raw_message)))
        
    
    def __str__(self):
        return f'Chuck: {self.chuck}, Checksum: {self.checksum}, Counter: {self.counter}'
 
class MessageConstructor:
    def __init__(self):
        self.counter = 0

    def checksum(self, message):
        return sum(ord(c) for c in message) % 2**16 # 16 bit checksum

    def create_message(self, message):
        chunks = [message[i:i+4] for i in range(0, len(message), 4)]# every 4 letters
        m_array = []
        for chunk in chunks:
            self.counter = (self.counter + 1) % 2**16 # 16 bit counter
            cs = self.checksum(chunk)
            #print(f'Sending chunk: {chunk}, Checksum: {cs}, Counter: {self.counter}')
            m = Message(chunk, cs, self.counter).get_struct()
            m_array.append(m)
        return m_array
    
def send_message(message):
    mc = MessageConstructor()
    m_array = mc.create_message(message)
    for m in m_array:
        new_message = canalystii.Message(can_id=0x300,
                                 remote=False,
                                 extended=False,
                                 data_len=8,
                                 data=(m))
        dev.send(new_message)
        
def dataInputBuffer(i):
    data = []
    try:
        data.append(dev.receive(i))
    except canalystii.NoData:
        pass
    
    return data

def test(message : str):
    mc = MessageConstructor()
    m_array = mc.create_message(message)
    reconstructed_message = ''
    for m in m_array:
        letters_received = Message.fromBytesToMessage(m)
        reconstructed_message += letters_received.chuck.decode('ascii')
        print('Sent: ', letters_received)
        # send_message(m)
        # print('Received: ', dataInputBuffer(1))
        data = dataInputBuffer(1)
        received_message = Message.fromBytesToMessage(data[0].data)
        print('Received: ', received_message)
        # check if the received message is correct by checking the checksum
        # perform checksum on the received message
        checksum = mc.checksum(received_message.chuck)
        if checksum == received_message.checksum:
            print('Checksum matches')
        else:
            print('Checksum does not match!!!')
        
    print('Original message: ', message)
    print('Reconstructed message: ', reconstructed_message)

In [26]:
message = 'Hello World'
test(message)

Sent:  Chuck: b'Hell', Checksum: 389, Counter: 1
Sent:  Chuck: b'o Wo', Checksum: 341, Counter: 2
Sent:  Chuck: b'rld\x00', Checksum: 322, Counter: 3
Original message:  Hello World
Reconstructed message:  Hello World 


In [27]:
#sending json
message = json.dumps({'name': 'John', 'age': 30})
test(message)


Sent:  Chuck: b'{"na', Checksum: 364, Counter: 1
Sent:  Chuck: b'me":', Checksum: 302, Counter: 2
Sent:  Chuck: b' "Jo', Checksum: 251, Counter: 3
Sent:  Chuck: b'hn",', Checksum: 292, Counter: 4
Sent:  Chuck: b' "ag', Checksum: 266, Counter: 5
Sent:  Chuck: b'e": ', Checksum: 225, Counter: 6
Sent:  Chuck: b'30}\x00', Checksum: 224, Counter: 7
Original message:  {"name": "John", "age": 30}
Reconstructed message:  {"name": "John", "age": 30} 
